# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.


## Open the data file and have a look at the general information. 


In [2]:
# Loading all the libraries
import pandas as pd

# Load the data
try:
    customer_info = pd.read_csv('credit_scoring_eng.csv')
except:
    customer_info = pd.read_csv('/datasets/credit_scoring_eng.csv') 
    


## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


In [3]:
# Let's see how many rows and columns our dataset has 

customer_info.shape

(21525, 12)

In [4]:
# let's print the first N rows

customer_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family



There are lots of issues to be fixed before processing the data such as lower and upper case use in education column, negative values in days_employed column, or different values in purpose column that mean the same thing(ex: purchase of the house or purchase of the house for my family). 

In [5]:
# Get info on data
customer_info.info()
customer_info.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

There are missing values in both days_employed and total_income column. 

In [6]:
# Let's look in the filtered table at the the first column with missing data
customer_info[customer_info['days_employed'].isna() == True]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


There's missing values do not seem symmetric as they don't have any patterns related to other columns. But I need to work on the data to make sure. 

In [7]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

customer_info.loc[(customer_info['days_employed'].isna() == True)&(customer_info['total_income'].isna() == True)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [11]:
customer_info.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Intermediate conclusion**

[Does the number of rows in the filtered table match the number of missing values? What conclusion can we make from this?]
Yes, it does. We can conclude that any row missing days_employed value is also missing total_income value or vice versa.


[Calculate the percentage of the missing values compared to the whole dataset. Is it a considerably large piece of data? If so, you may want to fill the missing values. To do that, firstly we should consider whether the missing data could be due to the specific client characteristic, such as employment type or something else. You will need to decide which characteristic *you* think might be the reason. Secondly, we should check whether there's any dependence missing values have on the value of other indicators with the columns with identified specific client characteristic.]


 10% (percentage of the missing values compared to the whole dataset)

[Explain your next steps and how they correlate with the conclusions you made so far.]


If a client is unemployed or a student, then they would not have a record of employment days or income. So, I would like to check if these missing values are correlated with employment type. 

In [7]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values

customer_info[customer_info['days_employed'].isna() == True]




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [8]:
# Checking distribution

df=customer_info[(customer_info['days_employed'].isna() == True)]
df['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

[Describe your findings here.]
The missing values seem to be from 5 income types above. I will check the distribution of income types in the whole data set to see if this gives us any pattern. 
**Possible reasons for missing values in data**

[Propose your ideas on why you think the values might be missing. Do you think they are missing randomly or there are any patterns?]
I do not think there is any pattern among missing values as they seem pretty random to me. It does not depend on any other column. It could be due to loss of information while recording or importing the data or incomplete data entry. 



In [9]:
# Checking the distribution in the whole dataset

customer_info['income_type'].value_counts()



employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

**Intermediate conclusion**

[Is the distribution in the original dataset similar to the distribution of the filtered table? What does that mean for us?]
Yes, it is. It means that there is no pattern on missing values as the distribtuion is similar.



In [10]:
# Check for other reasons and patterns that could lead to missing values
df_children=customer_info[(customer_info['days_employed'].isna() == True)]
df_children['children'].value_counts()

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

In [11]:
customer_info['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [12]:
df_family_status=customer_info[(customer_info['days_employed'].isna() == True)]
df_family_status['family_status'].value_counts()

married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64

In [13]:
customer_info['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

**Intermediate conclusion**

[Can we finally confirm that missing values are accidental? Check for anything else that you think might be important here.]
Yes, the missing values are accidental.

In [14]:
# Checking for other patterns - explain which

**Conclusions**

[Did you find any patterns? How did you come to this conclusion?]
No, I did not find any patterns on missing values. To come to this conclusion, I compared the data distribution on missing values with distribution of the whole dataset.


[Explain how you will address the missing values. Consider the categories in which values are missing.]
I will replace missing values with mean or median depending on a column that is more appropriate for the analysis. 



## Data transformation

[Let's go through each column to see what issues we may have in them.]


In [15]:
# Let's see all values in education column to check if and what spellings will need to be fixed
customer_info['education'].unique()


array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [16]:
# Fix the registers if required
customer_info['education'] = customer_info['education'].str.lower()

In [17]:
# Checking all the values in the column to make sure we fixed them

customer_info['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [18]:
# Let's see the distribution of values in the `children` column
customer_info['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

[Are there any strange things in the column? If yes, how high is the percentage of problematic data? How could they have occurred? Make a decision on what you will do with this data and explain you reasoning.]

The number of children cannot be equal to -1 and having 20 children is pretty uncommon. The percentage of problematic data is about 0.6%. These problems might have ocurred as a typo while entering the data.  I will replace problematic data with the mean as there is no significant outliners. 

In [19]:
customer_info[customer_info['children'] == 20].mean()

children               20.000000
days_employed       35779.535997
dob_years              41.815789
education_id            0.842105
family_status_id        0.815789
debt                    0.105263
total_income        26995.284209
dtype: float64

In [20]:
# [fix the data based on your decision]
customer_info['children'].mean()

0.5389082462253194

Since the mean is really close to 0, I decided to replace problematic values with 0 (which is also the median) instead of the mean. It would not make any difference as it's ony 0.6% of the whole dataset.

In [21]:
customer_info['children'] = customer_info['children'].replace([20, -1],0)


In [22]:
# Checking the `children` column again to make sure it's all fixed
customer_info['children'].value_counts()



0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

[Check the data in the `days_employed` column. Firstly think about what kind of issues could there be and what you may want to check and how you will do it.]

I saw some negative values in the days_employed column,so I would like to check that.

In [23]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
customer_info[customer_info['days_employed'] < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [24]:
(15906/21525) * 100 

73.89547038327527

73% of the days_employed column is problematic data, as a person cannot work for negative days. 

[If the amount of problematic data is high, it could've been due to some technical issues. We may probably want to propose the most obvious reason why it could've happened and what the correct data might've been, as we can't drop these problematic rows.]
The issue could be caused while the computer calculates the days worked. The values look pretty random so I don't think there is anything to do to fix them as they are not required in our analysis.

In [25]:
# Address the problematic values, if they exist



In [26]:
# Check the result - make sure it's fixed


[Let's now look at the client's age and whether there are any issues there. Again, think about what can data can be strange in this column, i.e. what cannot be someone's age.]

A person who is younger than 18 cannot take out a loan. 

In [27]:
# Check the `dob_years` for suspicious values and count the percentage
customer_info[customer_info['dob_years'] < 18]            


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,-2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,-1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,-1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing
20462,0,338734.868540,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20577,0,331741.271455,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21179,2,-108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


[Decide what you'll do with the problematic values and explain why.]

I will replace the problematic values with the median because there are significant outliners.

In [28]:
# Address the issues in the `dob_years` column, if they exist
customer_info['dob_years'].median()

42.0

In [29]:
customer_info['dob_years'] = customer_info['dob_years'].replace([0],43.0)

In [30]:
# Check the result - make sure it's fixed
customer_info[customer_info['dob_years'] == 0] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


[Now let's check the `family_status` column. See what kind of values there are and what problems you may need to address.]

In [31]:
# Let's see the values for the column
customer_info['family_status'].value_counts()


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

There are no problematic values in this column.

[Now let's check the `gender` column. See what kind of values there are and what problems you may need to address]

In [34]:
# Let's see the values in the column
customer_info['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

I will drop the problematc value since it is only one value. 

In [35]:
# Address the problematic values, if they exist
customer_info['gender'] = customer_info['gender'].replace(['XNA'],'NaN')

In [36]:
# Check the result - make sure it's fixed

customer_info['gender'].value_counts()

F      14236
M       7288
NaN        1
Name: gender, dtype: int64

[Now let's check the `income_type` column. See what kind of values there are and what problems you may need to address]

In [37]:
# Let's see the values in the column
customer_info['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

There is no problematic value in this column.

[Now let's see if we have any duplicates in our data. If we do, you'll need to decide what you will do with them and explain why.]

In [40]:
# Checking duplicates
customer_info.duplicated().sum()


71

I will drop the duplicates because it is vey unlikely to have 2 or more people with the exact same data. 

In [41]:
# Address the duplicates, if they exist
customer_info = customer_info.drop_duplicates().reset_index(drop=True)

In [42]:
# Last check whether we have any duplicates
customer_info.duplicated().sum()

0

In [43]:
# Check the size of the dataset that you now have after your first manipulations with it
customer_info.size

257448

[Describe your new dataset: briefly say what's changed and what's the percentage of the changes, if there were any.]
I dropped the duplicatesin the whole data set and changed all letters to lower case in education columns to make sure data has unique keys. I replaced the problematic value in dob_years and children column with appropriate data to not mess up the dataset. The percentage of the changes is (258300 - 257448)/258300 x 100= 0.3% which is the size of the data set at the beginning minus the size of the data set at the end divided by the begining size of the dataset.

# Working with missing values

[To speed up working with some data, you may want to work with dictionaries for some values, where IDs are provided. Explain why and which dictionaries you will work with.]
I will use dictionaries on the purpose column to keep the elements in key-value mapping format which will help ease the analysis.

In [44]:
customer_info['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [45]:
# Find the dictionaries
purpose_to_category = {
    'purchase of the house': 'house',
    'housing transactions': 'house',
    'purchase of the house for my family': 'house',
    'buy residential real estate': 'house',
    'construction of own property': 'house',
    'property': 'house',
    'building a property': 'house',
    'housing': 'house',
    'purchase of my own house': 'house',
    'housing renovation': 'house',
    'buying property for renting out': 'house',
    'buy real estate': 'real estate',
    'buy commercial real estate': 'real estate',
    'transactions with commercial real estate': 'real estate',
    'building a real estate': 'real estate',
    'transactions with my real estate': 'real estate',
    'real estate transactions': 'real estate',
    'car purchase': 'car',
    'buying a second-hand car': 'car',
    'buying my own car': 'car',
    'cars': 'car',
    'second-hand car purchase': 'car',
    'car': 'car',
    'to own a car': 'car', 
    'purchase of a car': 'car',
    'to buy a car': 'car',
    'supplementary education': 'education',
    'education': 'education',
    'to become educated': 'education',
    'getting an education': 'education',
    'to get a supplementary education': 'education',
    'getting higher education': 'education',
    'university education': 'education',
    'going to university': 'education',
    'to have a wedding': 'wedding',
    'having a wedding': 'wedding',
    'wedding ceremony': 'wedding',    
}

### Restoring missing values in `total_income`

[Briefly state which column(s) have values missing that you need to address. Explain how you will fix them.]
days_employed and total_income columns have missing values. I will replace the missing values with mean or median depending on whichever is more appropriate.



In [46]:
# Let's write a function that calculates the age category

def age_group(dob_years):
    if 18 < dob_years <= 24:
        return 'youth'
    if 25 <= dob_years <65:
        return 'adult'
    return 'senior'


In [47]:
# Test if the function works
age_group(68)

'senior'

In [48]:

# Creating new column based on function

customer_info['age_group'] = customer_info['dob_years'].apply(age_group)

In [49]:
# Checking how values in the new column

customer_info['age_group'].value_counts()

adult     19684
senior      895
youth       875
Name: age_group, dtype: int64

[Think about the factors on which income usually depends. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. To make this decision you will probably want to look at the distribution of the factors you identified as impacting one's income.]
Age, education, income type, or marriage status might affect income. 

In [50]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
customer_info_drop = customer_info[(customer_info['days_employed'].isna() == False)&(customer_info['total_income'].isna() == False)]
customer_info_drop.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,-4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,-5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,-4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
5,0,-926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult
6,0,-2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult
7,0,-152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,adult
8,2,-6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult
9,0,-2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult


In [51]:
# Look at the mean values for income based on your identified factors
array = ['age_group', 'education', 'family_status', 'income_type', 'gender']
for i in array:
    print(customer_info_drop.groupby(i)['total_income'].mean())
    print("\n")



age_group
adult     27208.428335
senior    21542.650450
youth     22703.351103
Name: total_income, dtype: float64


education
bachelor's degree      33142.802434
graduate degree        27960.024667
primary education      21144.882211
secondary education    24594.503037
some college           29045.443644
Name: total_income, dtype: float64


family_status
civil partnership    26694.428597
divorced             27189.354550
married              27041.784689
unmarried            26934.069805
widow / widower      22984.208556
Name: total_income, dtype: float64


income_type
business                       32386.793835
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64


gender
F      24655.604757
M      3

In [52]:
# Look at the median values for income based on your identified factors
array = ['age_group', 'education', 'family_status', 'income_type', 'gender']
for i in array:
    print(customer_info_drop.groupby(i)['total_income'].median())
    print("\n")

age_group
adult     23540.1295
senior    18471.3910
youth     20572.2090
Name: total_income, dtype: float64


education
bachelor's degree      28054.5310
graduate degree        25161.5835
primary education      18741.9760
secondary education    21836.5830
some college           25618.4640
Name: total_income, dtype: float64


family_status
civil partnership    23186.534
divorced             23515.096
married              23389.540
unmarried            23149.028
widow / widower      20514.190
Name: total_income, dtype: float64


income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64


gender
F      21464.845
M      26834.295
NaN    32624.825
Name: total_income, dtyp

[Repeat such comparisons for multiple factors. Make sure you consider different aspects and explain your thinking process.]
I compared multiple conditions which are age_group, education, family_status, children, and gender to get different aspects of the data. My logic was that as you get older your experience level increases which directly affects your income. So does education level. I did not think family status or number of children would affect the income but only the the amount of tax they are paying, thus I wanted to make sure. Lastly, I wanted to consider economic inequality by gender and grouped the data by gender column as well. 


[Make a decision on what characteristics define income most and whether you will use a median or a mean. Explain why you made this decision]
I will use median to replace the missing values because we need to take the outliners into account in this situation. In the dataset, there are very low or high incomes which end up in a higher mean value. The characteristics that affect the income most are income types, education, and gender. But I will use education to fill in missing values as I believe it to be more accurate.

In [57]:
#  Write a function that we will use for filling in missing values
income_groups_missing_values = ["bachelor's degree", 'graduate degree', 'primary education', 'secondary education', 'some college'] 

for group in income_groups_missing_values:
    group_median_value = customer_info[customer_info['income_type'] == group]['total_income'].median()
    customer_info.loc[(customer_info['income_type'] == group), 'total_income'] = customer_info.loc[(customer_info['income_type'] == group), 'total_income'].fillna(group_median_value)

In [58]:
# Check if it works

customer_info['total_income'].isna().sum()

2103

In [243]:
# Apply it to every row
#customer_info['total_income'] = customer_info['education'].apply(missing_income, axis=1)
customer_info

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,-4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,-5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,-4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43.0,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,adult
21450,0,343937.404131,67.0,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,senior
21451,1,-2113.346888,38.0,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult
21452,3,-3112.481705,38.0,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adult


[If you've came across errors in preparing the values for missing data, it probably means there's something special about the data for the category. Give it some thought - you may want to fix some things manually, if there's enough data to find medians/means.]


In [245]:
# Checking the number of entries in the columns

len(customer_info['total_income'])

21454

###  Restoring values in `days_employed`

In [246]:
# Distribution of `days_employed` medians based on your identified parameters

array = ['age_group', 'education', 'family_status', 'income_type', 'gender']
for i in array:
    print(customer_info_drop.groupby(i)['days_employed'].median())
    print("\n")


age_group
adult      -1329.335740
senior    360304.232308
youth       -744.016267
Name: days_employed, dtype: float64


education
bachelor's degree     -1342.373432
graduate degree       -1380.316041
primary education      -551.062561
secondary education   -1184.327177
some college          -1046.437583
Name: days_employed, dtype: float64


family_status
civil partnership     -1197.176853
divorced              -1146.122484
married               -1332.196271
unmarried             -1015.245549
widow / widower      337017.713307
Name: days_employed, dtype: float64


income_type
business                        -1547.382223
civil servant                   -2689.368353
employee                        -1574.202821
entrepreneur                     -520.848083
paternity / maternity leave     -3296.759962
retiree                        365213.306266
student                          -578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64


gender
F     -1179.3

In [247]:
# Distribution of `days_employed` means based on your identified parameters
array = ['age_group', 'education', 'family_status', 'income_type', 'gender']
for i in array:
    print(customer_info_drop.groupby(i)['days_employed'].mean())
    print("\n")


age_group
adult      54318.069835
senior    312933.276967
youth       -423.178637
Name: days_employed, dtype: float64


education
bachelor's degree       38323.055702
graduate degree        116630.048157
primary education      127842.088750
secondary education     72538.686698
some college            17692.508357
Name: days_employed, dtype: float64


family_status
civil partnership     54587.019762
divorced              64819.140232
married               59202.282275
unmarried             43834.346331
widow / widower      202957.656283
Name: days_employed, dtype: float64


income_type
business                        -2111.524398
civil servant                   -3399.896902
employee                        -2326.499216
entrepreneur                     -520.848083
paternity / maternity leave     -3296.759962
retiree                        365003.491245
student                          -578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64


gender
F  

[Decide what you will use: means or medians. Explain why.]
I will use medians to replace missing values because the mean gives negative values as 73% of the data. 

In [248]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
def calculate_mean(indetifier):
    return customer_info.groupby(indetifier)['days_employed'].mean()

In [249]:
# Check that the function works
calculate_mean('education')


education
bachelor's degree       38323.055702
graduate degree        116630.048157
primary education      127842.088750
secondary education     72538.686698
some college            17692.508357
Name: days_employed, dtype: float64

In [250]:
# Apply function to the income_type
#customer_info_income=float(customer_info['income_type'])
#customer_info_income.apply(calculate_mean)

calculate_mean('income_type')

income_type
business                        -2111.524398
civil servant                   -3399.896902
employee                        -2326.499216
entrepreneur                     -520.848083
paternity / maternity leave     -3296.759962
retiree                        365003.491245
student                          -578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

In [251]:
# Check if function worked
customer_info['income_type']

0        employee
1        employee
2        employee
3        employee
4         retiree
           ...   
21449    business
21450     retiree
21451    employee
21452    employee
21453    employee
Name: income_type, Length: 21454, dtype: object

In [252]:
# Replacing missing values
customer_info['days_employed'].fillna(customer_info['days_employed'].mean())


0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21449     -4529.316663
21450    343937.404131
21451     -2113.346888
21452     -3112.481705
21453     -1984.507589
Name: days_employed, Length: 21454, dtype: float64

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [253]:
# Check the entries in all columns - make sure we fixed all missing values
len(customer_info['days_employed'])

21454

In [254]:
customer_info.shape

(21454, 13)

## Categorization of data

[To answer the questions and test the hypotheses, you will want to work with categorized data. Look at the questions that were posed to you and that you should answer. Think about which of the data will need to be categorized to answer these questions. Below you will find a template through which you can work your way when categorizing data. The first step-by-step processing covers the text data; the second one addresses the numerical data that needs to be categorized. You can use both or none of the suggested instructions - it's up to you.]

[Despite of how you decide to address the categorization, make sure to provide clear explanation of why you made your decision. Remember: this is your work and you make all decisions in it.]


In [255]:
# Print the values for your selected data for categorization

customer_info['children']


0        1
1        1
2        0
3        3
4        0
        ..
21449    1
21450    0
21451    1
21452    3
21453    2
Name: children, Length: 21454, dtype: int64

In [256]:
# Check the unique values
customer_info['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [257]:
# Let's write a function to categorize the data based on common topics
def categorize(children):
    if 0 <= children <= 2:
        return 'small family'
    if 2 < children <= 4:
        return 'average family'
    return 'large family'

In [258]:
# Create a column with the categories and count the values for them
customer_info['family_size'] = customer_info['children'].apply(categorize)
customer_info['family_size'].value_counts()

small family      21074
average family      371
large family          9
Name: family_size, dtype: int64

In [259]:
# Looking through all the numerical data in your selected column for categorization
customer_info['family_size']

0          small family
1          small family
2          small family
3        average family
4          small family
              ...      
21449      small family
21450      small family
21451      small family
21452    average family
21453      small family
Name: family_size, Length: 21454, dtype: object

In [260]:
# Getting summary statistics for the column
customer_info.groupby('family_size')['children'].sum()


family_size
average family    1154
large family        45
small family      8912
Name: children, dtype: int64

[Decide what ranges you will use for grouping and explain why.]
Having 0-2 children would be a small family where as having 3-4 children is average. Having more than 4 children is considered a large family. 

In [264]:
customer_info['income_type'].value_counts()

employee                       11084
business                        5078
retiree                         3829
civil servant                   1457
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [265]:
def categorize_income(income_type):
    if income_type == 'employee':
        return 'employee'
    if income_type == 'business':
        return 'business'
    if income_type == 'retiree':
        return 'retiree'
    if income_type == 'civil servant':
        return 'civil servant'
    return 'other'

In [266]:
customer_info['income_groups'] = customer_info['income_type'].apply(categorize_income)
customer_info['income_groups'].value_counts()

employee         11084
business          5078
retiree           3829
civil servant     1457
other                6
Name: income_groups, dtype: int64

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [267]:
customer_info.pivot_table(index='children', values='debt', aggfunc=['sum', 'count', 'mean'])

,sum,count,mean
,debt,debt,debt
children,,,
0,1072,14214,0.075419
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


**Conclusion**

[Write your conclusions based on your manipulations and observations.]
Around 70-90% of the people in the dataset struggle to pay back on time depending on the number of children they have.

**Is there a correlation between family status and paying back on time?**

In [268]:
customer_info.pivot_table(index='family_status', values='debt', aggfunc=['sum', 'count', 'mean'])

,sum,count,mean
,debt,debt,debt
family_status,,,
civil partnership,388,4151,0.093471
divorced,85,1195,0.071130
married,931,12339,0.075452
unmarried,274,2810,0.097509
widow / widower,63,959,0.065693


**Conclusion**

[Write your conclusions based on your manipulations and observations.]
Widowers are more likely to pay back on time whereas unmarried people more likely to have a debt.

**Is there a correlation between income level and paying back on time?**

In [269]:
customer_info.pivot_table(index='income_groups', values='debt', aggfunc=['sum', 'count', 'mean'])


,sum,count,mean
,debt,debt,debt
income_groups,,,
business,376,5078,0.074045
civil servant,86,1457,0.059025
employee,1061,11084,0.095724
other,2,6,0.333333
retiree,216,3829,0.056412


**Conclusion**

[Write your conclusions based on your manipulations and observations.]
People who are employees are more likely to carry a debt, and retirees tend to carry les debt according to the dataset.

**How does credit purpose affect the default rate?**

In [270]:
# Check the percentages for default rate for each credit purpose and analyze them
purposes=['car','house','education','wedding','real estate']
for i in purposes:
    count_no_debt=len(customer_info[(customer_info['purpose'].str.contains(i)) & (customer_info['debt'] == 0)])
    count_debt=len(customer_info[(customer_info['purpose'].str.contains(i)) & (customer_info['debt'] == 1)])
    print("ratio for "+i+" buyers : "+ str(count_debt/count_no_debt))
    


ratio for car buyers : 0.1032539072508327
ratio for house buyers : 0.0714687675858188
ratio for education buyers : 0.10209145693016661
ratio for wedding buyers : 0.08699719363891488
ratio for real estate buyers : 0.08139534883720931


In [271]:
customer_info.pivot_table(index='purpose', values='debt', aggfunc=['sum', 'count', 'mean'])

,sum,count,mean
,debt,debt,debt
purpose,,,
building a property,54,619,0.087237
building a real estate,48,624,0.076923
buy commercial real estate,47,661,0.071104
buy real estate,43,621,0.069243
buy residential real estate,41,606,0.067657
buying a second-hand car,36,478,0.075314
buying my own car,46,505,0.091089
buying property for renting out,52,651,0.079877


**Conclusion**

[Write your conclusions based on your manipulations and observations.]


House buyers are more likely to pay back on time with 93% followed by real estate, wedding, education, and cars buyers. 

# General Conclusion 


I started off with determining how to preprocess the data before analyzing it. Getting rid off the problematic data by either replacing or dropping the values was the first step. Then, I dropped all the duplicates to have a clean data. To fill in missing values, I needed to categorize the data based on paramaters I thought was necessary which helped me to decide whether to use means or medians to fill in. I couldn't find any patterns on missing values and they seem accidental. The problem might be due to incomplete data entry, or losing data while importing it. Lastly, to check the hypothesis, I determined if there is any correlation between paying back on time and conditions such as number of children, income type, or family status. Around 70-90% of the people in the dataset struggle to pay back on time depending on the number of children they have. People who are employees are more likely to carry a debt, and retirees tend to carry les debt according to the dataset. Widowers are more likely to pay back on time whereas unmarried people more likely to have a debt.